
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/3_Prepare_Data_Partition_LabelMap_CSVtoXML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/3_Prepare_Data_Partition_LabelMap_CSVtoXML.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


### Import necessary libraries

In [1]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import * 
!wget {XML_TO_CSV}
!wget {PARTITION}

from xml_to_csv import *
from partition import *
from PIL import Image
import os
import shutil


--2020-06-10 21:44:58--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4839 (4.7K) [text/plain]
Saving to: ‘constants.py’

constants.py        100%[===================>]   4.73K  --.-KB/s    in 0s      

2020-06-10 21:44:58 (69.3 MB/s) - ‘constants.py’ saved [4839/4839]

--2020-06-10 21:44:59--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/xml_to_csv.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1261 

### Prepare dataset ###

#### Connect to Google Drive

In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Copy dataset from Google Drive and extract


In [3]:
%cp {DRIVE_DEV}/unmasked-datasetv2.zip /
%cp {DRIVE_DEV}/masked-datasetv2.zip /
%cd /
!unzip -o /unmasked-datasetv2.zip 
!unzip -o /masked-datasetv2.zip 
%cd /content/

/
Archive:  /unmasked-datasetv2.zip
  inflating: content/tensorflow/workspace/Face-Mask-Detection/dataset/unprepared/images/1HJCAS6U4B.jpg  
  inflating: content/tensorflow/workspace/Face-Mask-Detection/dataset/unprepared/images/2HWQ2T1E3O.jpg  
  inflating: content/tensorflow/workspace/Face-Mask-Detection/dataset/unprepared/images/03EZPUXP7Y.jpg  
  inflating: content/tensorflow/workspace/Face-Mask-Detection/dataset/unprepared/images/2RUF1V05NB.jpg  
  inflating: content/tensorflow/workspace/Face-Mask-Detection/dataset/unprepared/images/2W8WZ3XVFE.jpg  
  inflating: content/tensorflow/workspace/Face-Mask-Detection/dataset/unprepared/images/4BHJRM78NR.jpg  
  inflating: content/tensorflow/workspace/Face-Mask-Detection/dataset/unprepared/images/4MHTUTPWQ3.jpg  
  inflating: content/tensorflow/workspace/Face-Mask-Detection/dataset/unprepared/images/3R4A12KY9R.jpg  
  inflating: content/tensorflow/workspace/Face-Mask-Detection/dataset/unprepared/images/4DPX6K8KVO.jpg  
  inflating: conten

### Partitioning the images

We continue our data preparation by splitting our dataset into training and test sets.

Typically, the ratio is 90%/10%, i.e. 90% of the images are used for training and the rest 10% is maintained for testing, but you can chose whatever ratio suits your needs.


In [4]:
ratio = 0.1
partition_dir(DATASET_DIR_UNPREP_IMG, DATASET_DIR_UNPREP_ANNO,  DATASET_DIR_PREP_TRAIN_IMG , DATASET_DIR_PREP_TRAIN_ANNO, DATASET_DIR_PREP_TEST_IMG, DATASET_DIR_PREP_TEST_ANNO, ratio) 

191


#### Delete original files (Optional) 

Once we are happy the images have successfully split into training and test sets, we can go ahead and delete the original images.

In [0]:
!rm -r {DATASET_DIR_UNPREP}

### Label Map

TensorFlow requires a label map, which namely maps each of the used labels to an integer values. This label map is used both by the training and detection processes.

In [5]:
!wget {LABEL_MAP} -P {DATASET_DIR_PREP_RECORD}

--2020-06-10 21:45:37--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/label_map.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79 [text/plain]
Saving to: ‘/content/tensorflow/workspace/Face-Mask-Detection/dataset/prepared/record/label_map.pbtxt’

label_map.pbtxt     100%[===================>]      79  --.-KB/s    in 0s      

2020-06-10 21:45:37 (4.53 MB/s) - ‘/content/tensorflow/workspace/Face-Mask-Detection/dataset/prepared/record/label_map.pbtxt’ saved [79/79]



### XML to CSV

And finally we need to generate a CSV file containing all image detail / classes from the individual XML files.

#### Generate CSV files

For the train and test folders and place result in annotations folder

In [6]:
generate_csv(DATASET_DIR_PREP_TRAIN_ANNO, DATASET_DIR_PREP_RECORD )
generate_csv(DATASET_DIR_PREP_TEST_ANNO, DATASET_DIR_PREP_RECORD )

Successfully converted train xml to csv.
Successfully converted test xml to csv.


### Save

#### Compress Dataset

Follow this step if you wish to archive your dataset for future use

In [0]:
!zip -r partitioned-datasetv2.zip {DATASET_DIR_PREP}
!gsutil cp partitioned-datasetv2.zip {DRIVE_DEV}

  adding: content/training_demo/ (stored 0%)
  adding: content/training_demo/annotations/ (stored 0%)
  adding: content/training_demo/annotations/label_map.pbtxt (deflated 42%)
  adding: content/training_demo/annotations/train_labels.csv (deflated 72%)
  adding: content/training_demo/annotations/test_labels.csv (deflated 68%)
  adding: content/training_demo/images/ (stored 0%)
  adding: content/training_demo/images/train/ (stored 0%)
  adding: content/training_demo/images/train/2OKEGQH2XR.jpg (deflated 0%)
  adding: content/training_demo/images/train/3QOQ2TMK9U.jpg (deflated 0%)
  adding: content/training_demo/images/train/0G8BKB200W.jpg (deflated 0%)
  adding: content/training_demo/images/train/JRWBZ0KMDZ.jpg (deflated 1%)
  adding: content/training_demo/images/train/4B5AC7FB7W.jpg (deflated 0%)
  adding: content/training_demo/images/train/0NZVBZ17MN.jpg (deflated 0%)
  adding: content/training_demo/images/train/4VT0GB9FOM.xml (deflated 47%)
  adding: content/training_demo/images/trai

Copying file://partitioned-dataset.zip...
/ [1 files][ 34.2 MiB/ 34.2 MiB]                                                
Operation completed over 1 objects/34.2 MiB.                                     
